In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import save_image
import os
import numpy as np

In [ ]:
batch_size = 128
epochs = 300
latent_dim = 100
channels = 3
image_size = 64
learning_rate = 0.0002
beta1 = 0.5

In [ ]:
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

anime_faces = datasets.ImageFolder("/kaggle/input/animefacedataset/", transform=transform)
dataloader = DataLoader(anime_faces, batch_size=batch_size, shuffle=True, num_workers=4)

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # Input is a latent_dim-dimensional noise
            nn.ConvTranspose2d(latent_dim, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            # Output size: (512, 4, 4)
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            # Output size: (256, 8, 8)
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            # Output size: (128, 16, 16)
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            # Output size: (64, 32, 32)
            nn.ConvTranspose2d(64, channels, 4, 2, 1, bias=False),
            nn.Tanh()
            # Output size: (channels, 64, 64)
        )

    def forward(self, input):
        return self.main(input)

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            # Input size: (channels, 64, 64)
            nn.Conv2d(channels, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # Output size: (64, 32, 32)
            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
             # Output size: (128, 16, 16)
            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            # Output size: (256, 8, 8)
            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            # Output size: (512, 4, 4)
            nn.Conv2d(512, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
            # Output size: (1, 1, 1)
        )

    def forward(self, input):
        return self.main(input).view(-1, 1).squeeze(1)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

generator = Generator()
discriminator = Discriminator()
generator.load_state_dict(torch.load('/kaggle/input/300-epoch/generator.pth'))
discriminator.load_state_dict(torch.load('/kaggle/input/300-epoch/discriminator.pth'))
generator = generator.to(device)
discriminator = discriminator.to(device)

optimizer_G = optim.Adam(generator.parameters(), lr=learning_rate, betas=(beta1, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=learning_rate, betas=(beta1, 0.999))

criterion = nn.BCELoss()

In [ ]:
os.makedirs("/kaggle/working/output", exist_ok=True)

fixed_noise = torch.randn(64, latent_dim, 1, 1, device=device)

for epoch in range(epochs):
    for i, (real_images, _) in enumerate(dataloader):
        batch_size = real_images.size(0)
        real_images = real_images.to(device)

        # Train the discriminator
        optimizer_D.zero_grad()
        real_labels = torch.full((batch_size,), 1, device=device, dtype=torch.float)
        real_output = discriminator(real_images)
        d_loss_real = criterion(real_output, real_labels)
        d_loss_real.backward()

        noise = torch.randn(batch_size, latent_dim, 1, 1, device=device)
        fake_images = generator(noise)
        fake_labels = torch.full((batch_size,), 0, device=device, dtype=torch.float)
        fake_output = discriminator(fake_images.detach())
        d_loss_fake = criterion(fake_output, fake_labels)
        d_loss_fake.backward()

        d_loss = d_loss_real + d_loss_fake
        optimizer_D.step()

        # Train the generator
        optimizer_G.zero_grad()
        output = discriminator(fake_images)
        g_loss = criterion(output, real_labels)
        g_loss.backward()
        optimizer_G.step()

    # Save generated images
    if (epoch + 1) % 10 == 0:
        with torch.no_grad():
            fake = generator(fixed_noise).detach().cpu()
        save_image(fake, f"/kaggle/working/output/fake_{epoch + 1}.png", nrow=8, normalize=True)

    print(f"[Epoch {epoch+1}/{epochs}] D_loss: {d_loss.item():.4f} G_loss: {g_loss.item():.4f}")

In [ ]:
torch.save(generator.state_dict(), 'generator.pth')
torch.save(discriminator.state_dict(), 'discriminator.pth')

## Task 2

In [ ]:
import matplotlib.pyplot as plt

def show_images(images, nrow=8):
    images = images.detach().cpu() 
    images = images / 2 + 0.5  
    images = images.numpy()
    images = np.transpose(images, (0, 2, 3, 1)) # adjust shape to (batch_size, image_size, image_size, channels)
    grid_img = np.concatenate([np.concatenate([images[i * nrow + j] for j in range(nrow)], axis=1)
                               for i in range(batch_size // nrow)], axis=0)
    plt.figure(figsize=(10, 10))
    plt.imshow(grid_img)
    plt.axis('off')
    plt.show()

In [ ]:
from torch.distributions import Normal, Beta, Poisson, Gamma

distributions = [Normal(0, 1), Beta(0.5, 0.5), Poisson(1), Gamma(1, 1)]

In [ ]:
print(distributions[2])

In [ ]:
for dist in distributions:
    noise = dist.sample((batch_size, latent_dim, 1, 1)).to(device)
    generated_faces = generator(noise)
    print(dist)
    show_images(generated_faces)

## Task 3 & 4

In [ ]:
# Suppose O1 and O2 are two real anime faces randomly taken from the dataset
O1 = anime_faces[0][0].unsqueeze(0).to(device)
O2 = anime_faces[1][0].unsqueeze(0).to(device)

def find_input(O):
    I = torch.randn((1, latent_dim, 1, 1), requires_grad=True, device=device)
    optimizer = optim.Adam([I], lr=0.01)

    for _ in range(1000):
        optimizer.zero_grad()
        O_hat = generator(I)
        loss = ((O_hat - O) ** 2).mean()
        loss.backward()
        optimizer.step()

    return I.detach()

I1 = find_input(O1)
I2 = find_input(O2)

In [ ]:
def show_single_image(image):
    image = image.detach().cpu()
    image = image / 2 + 0.5
    image = image.numpy() 
    image = np.transpose(image, (1, 2, 0))
    plt.imshow(image)
    plt.axis('off')
    plt.show()

In [ ]:
show_single_image(O1.squeeze(0))

In [ ]:
show_single_image(O2.squeeze(0))

In [ ]:
show_single_image(generator(I1).squeeze(0))

In [ ]:
show_single_image(generator(I2).squeeze(0))

In [ ]:
I_prime = (I1 + I2) / 2
O_prime = generator(I_prime)
show_single_image(O_prime.squeeze(0))

## Training resnet to learn inverse mapping of my generator

In [ ]:
from torchvision.models import resnet18

class InverseModel(nn.Module):
    def __init__(self):
        super(InverseModel, self).__init__()
        self.model = resnet18(pretrained=False)
        self.model.fc = nn.Linear(512, latent_dim)  # adjust output size

    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
inverse_model = InverseModel().to(device)
optimizer_I = optim.Adam(inverse_model.parameters(), lr=learning_rate, betas=(beta1, 0.999))

In [ ]:
criterion = nn.MSELoss()

In [ ]:
def generate_latent_vectors(images):
    images = images.to(device)
    latent_vectors = torch.randn(images.size(0), latent_dim, 1, 1, device=device, requires_grad=True)
    optimizer = torch.optim.Adam([latent_vectors], lr=0.1)

    for step in range(2000): 
        optimizer.zero_grad()
        generated_images = generator(latent_vectors)
        loss = ((generated_images - images)**2).mean()
        loss.backward()
        optimizer.step()

    return latent_vectors.detach()

In [ ]:
for epoch in range(epochs):
    for i, (real_images, _) in enumerate(dataloader):
        real_images = real_images.to(device)

        real_latents = generate_latent_vectors(real_images).to(device)

        optimizer_I.zero_grad()
        predicted_latents = inverse_model(real_images)
        i_loss = criterion(predicted_latents, real_latents)
        i_loss.backward()
        optimizer_I.step()

    print(f"[Epoch {epoch+1}/{epochs}] I_loss: {i_loss.item():.4f}")

In [ ]:
torch.save(inverse_model.state_dict(),'inverse_model.pth')

In [ ]:
O1 = O1.to(device)
O2 = O2.to(device)
I1 = inverse_model(O1)
I2 = inverse_model(O2)

In [ ]:
I_prime = (I1 + I2) / 2
O_prime = generator(I_prime)

In [ ]:
save_image(O_prime, f"/kaggle/working/output/O_prime.png", nrow=8, normalize=True)